# Xebicon '19

Disponibilité des Velib dans la ville de Paris.

Basé surl'OpenData de la ville Paris : https://opendata.paris.fr/pages/home/

En utilisant le dataset Velib datasets : https://opendata.paris.fr/explore/dataset/velib-disponibilite-en-temps-reel/information/

In [27]:
#!pip install -r ../requirements.txt

In [28]:
import pandas as pd

In [29]:
stations = pd.read_csv("https://opendata.paris.fr/api/v2/catalog/datasets/velib-disponibilite-en-temps-reel/exports/csv?rows=-1&timezone=UTC&delimiter=%3B", delimiter=";")

In [30]:
stations[stations['station_name'].str.contains("Mongolfier")]

,nbfreeedock,nbbikeoverflow,nbedock,overflowactivation,densitylevel,creditcard,station,maxbikeoverflow,kioskstate,nbfreedock,...,overflow,nbdock,nbebike,nbebikeoverflow,station_code,station_name,station_state,station_type,geo,duedate
920,21,0,39,no,0,no,"{""code"": ""44101"", ""name"": ""Place Mongolfier"", ...",0,yes,0,...,no,0,7,0,44101,Place Mongolfier,Operative,yes,"48.8151600301, 2.45928794146",2018-10-30


In [31]:
stations.describe()

,nbfreeedock,nbbikeoverflow,nbedock,densitylevel,maxbikeoverflow,nbfreedock,nbbike,nbdock,nbebike,nbebikeoverflow,station_code
count,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000
mean,16.549640,0.090647,25.396403,0.902878,16.276259,3.750360,5.446763,5.607914,4.842446,0.012230,18038.243165
std,12.935741,0.403885,16.005817,0.296231,18.383756,9.347941,5.695764,12.949793,3.904041,0.128098,12068.090148
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1001.000000
25%,6.000000,0.000000,18.000000,1.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,11003.250000
50%,16.000000,0.000000,25.000000,1.000000,11.000000,0.000000,4.000000,0.000000,4.000000,0.000000,16010.500000
75%,24.000000,0.000000,35.000000,1.000000,30.000000,1.000000,8.000000,1.000000,7.000000,0.000000,21118.750000
max,69.000000,11.000000,73.000000,1.000000,71.000000,57.000000,44.000000,70.000000,27.000000,2.000000,92008.000000


In [32]:
stations[stations['nbebike'] == stations.describe()['nbebike']['max']]

,nbfreeedock,nbbikeoverflow,nbedock,overflowactivation,densitylevel,creditcard,station,maxbikeoverflow,kioskstate,nbfreedock,...,overflow,nbdock,nbebike,nbebikeoverflow,station_code,station_name,station_state,station_type,geo,duedate
722,20,0,51,no,1,no,"{""code"": ""11110"", ""name"": ""Roquette - Folie Re...",53,yes,0,...,no,1,27,0,11110,Roquette - Folie Regnault,Operative,yes,"48.8592469592, 2.38655362358",2018-10-25


In [33]:
stations[['lat','lng']] = stations['geo'].str.split(',',expand=True)
stations[['lat','lng', 'nbfreeedock']]
stations = stations.astype({'lng': 'float64', 'lat': 'float64'})
stations.dtypes

nbfreeedock             int64
nbbikeoverflow          int64
nbedock                 int64
overflowactivation     object
densitylevel            int64
                       ...   
station_type           object
geo                    object
duedate                object
lat                   float64
lng                   float64
Length: 23, dtype: object

In [34]:
stations.columns

Index(['nbfreeedock', 'nbbikeoverflow', 'nbedock', 'overflowactivation',
       'densitylevel', 'creditcard', 'station', 'maxbikeoverflow',
       'kioskstate', 'nbfreedock', 'nbbike', 'overflow', 'nbdock', 'nbebike',
       'nbebikeoverflow', 'station_code', 'station_name', 'station_state',
       'station_type', 'geo', 'duedate', 'lat', 'lng'],
      dtype='object')

Configure Google Maps API Key: https://jupyter-gmaps.readthedocs.io/en/latest/authentication.html

In [35]:
import os
import gmaps
import gmaps.datasets
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])  # Your Google API key

#print("Google Api Key:", os.environ["GOOGLE_API_KEY"])

In [36]:
paris_coordinates = (48.8566, 2.3522)



In [37]:
stations_with_almost_no_ebikes = stations[stations['nbebike'] < 4]

fig = gmaps.figure(center=paris_coordinates, zoom_level=13)

almost_no_ebikes_heatmap_layer = gmaps.heatmap_layer(stations_with_almost_no_ebikes[['lat', 'lng']])#
almost_no_ebikes_heatmap_layer.gradient = [(200, 200, 200, 0.0), 'yellow', 'red']
#almost_no_ebikes_heatmap_layer.max_intensity = 1
almost_no_ebikes_heatmap_layer.point_radius = 32

fig.add_layer(almost_no_ebikes_heatmap_layer)
#fig.add_layer(gmaps.bicycling_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
stations_with_many_ebikes = stations[stations['nbebike'] > 5]

fig = gmaps.figure(center=paris_coordinates, zoom_level=13)

many_ebikes_heatmap_layer = gmaps.heatmap_layer(stations_with_many_ebikes[['lat', 'lng']])#
many_ebikes_heatmap_layer.gradient = [(200, 200, 200, 0.0), 'yellow', 'green']
#many_ebikes_heatmap_layer.max_intensity = 2
many_ebikes_heatmap_layer.point_radius = 32

fig.add_layer(many_ebikes_heatmap_layer)
#fig.add_layer(gmaps.bicycling_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [39]:

fig = gmaps.figure(center=paris_coordinates, zoom_level=13)

fig.add_layer(almost_no_ebikes_heatmap_layer)
fig.add_layer(many_ebikes_heatmap_layer)
#fig.add_layer(gmaps.bicycling_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [40]:
paris_coordinates = (48.8566, 2.3522)

fig = gmaps.figure(center=paris_coordinates, zoom_level=13)
heatmap_layer = gmaps.heatmap_layer(stations[['lat', 'lng']], weights=stations['nbebike'])
heatmap_layer.gradient = [(200, 200, 200, 0.0), 'yellow', 'green']
heatmap_layer.point_radius = 32

fig.add_layer(heatmap_layer)
fig.add_layer(gmaps.bicycling_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
fig = gmaps.figure(center=paris_coordinates, zoom_level=13)
heatmap_layer = gmaps.heatmap_layer(stations[['lat', 'lng']], weights=stations['nbfreeedock'])
heatmap_layer.gradient = [(200, 200, 200, 0.0), 'yellow', 'green']
heatmap_layer.point_radius = 32

print(heatmap_layer.gradient)
fig.add_layer(heatmap_layer)
fig.add_layer(gmaps.bicycling_layer())

fig

['rgba(200,200,200,0.0)', 'yellow', 'green']


Figure(layout=FigureLayout(height='420px'))

In [42]:
no_ebike = stations[stations['nbebike'] == 0]
almost_no_ebike = stations[(stations['nbebike'] > 0) & (stations['nbebike'] < 4)]
ebikes = stations[stations['nbebike'] > 3]

In [43]:
scale = 3

no_ebike_layer = gmaps.symbol_layer(
    no_ebike[['lat', 'lng']], fill_color='red',
    stroke_color='red', scale=scale
)

almost_no_ebike_layer = gmaps.symbol_layer(
    almost_no_ebike[['lat', 'lng']], fill_color='yellow',
    stroke_color='yellow', scale=scale
)

ebikes_layer = gmaps.symbol_layer(
    ebikes[['lat', 'lng']], fill_color='green',
    stroke_color='green', scale=scale
)

info_box_template = """
<dl>
<dt>Name</dt><dd>{station_name}</dd>
<dt>Nb ebikes</dt><dd>{nbebike}</dd>
<dt>Nb free docks</dt><dd>{nbfreeedock}</dd>
</dl>
"""
station_info = [info_box_template.format(**station) for station in stations[['station_name', 'nbebike', 'nbfreeedock']].to_dict('records')]

marker_layer = gmaps.marker_layer(stations[['lat', 'lng']], info_box_content=station_info)

fig = gmaps.figure(center=paris_coordinates, zoom_level=12)

fig.add_layer(no_ebike_layer)
fig.add_layer(almost_no_ebike_layer)
fig.add_layer(ebikes_layer)
#fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))